In [1]:
import os, pyodbc
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine


In [2]:
%pylab


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\python27\arcgis10.2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Connect to SQL db and create dataframe

In [3]:
engine = create_engine("mssql+pyodbc://gt_KL_Master")


Query for Au standards

In [4]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL \
and e.DateEmis > '2017-1-1' and c.Reference <> 'CM24'  and isnumeric(Col101) = 1"

Upper Canada Standards

In [4]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL \
and a.NomProjet = 'UpperCanada'  and isnumeric(Col101) = 1"

Query for  CU standards

In [53]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col98 as 'Cu_ppm', c.Col99 as 'Cu_pct', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col98 is not NULL \
and e.DateEmis > '2017-1-1'"

Query for lab standards

In [6]:
sql = "Select  b.Nom as 'HOLEID', '0000' as 'SAMPLEID', c.Reference as 'STD', c.Col101 as 'Au_gpt', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQCLab c on b.idkey = c.idNuSondage \
join tblAnalRegCertificat e on c.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL and e.DateEmis > '2017-1-1'"

Query for Blanks

In [4]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date'\
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 1 and c.Col101 is not NULL \
and e.DateEmis > DATEADD(MONTH, -3, GETDATE()) \
and e.idLaboratoire = 6"

In [11]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date'\
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 1 and c.Col101 is not NULL \
and a.NomProjet = 'UpperCanada' and e.idLaboratoire = 6"

In [32]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date'\
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 1 and c.Col101 is not NULL \
and e.DateEmis between '2017-01-01' and '2017-07-01' and e.idLaboratoire = 6"

In [5]:
df = pd.read_sql(sql, engine) #, index_col = 'Date')

In [6]:
stds = pd.read_sql("Select * from tblTypeStandard", engine)
stds.columns = ['idkey','std_name','alias','col','avg','active','sigma']

In [7]:
if 'Au_gpt' in df.columns:
    df['Au_val'] = df.Au_gpt[(df.Au_gpt <> ' NSS') & (df.Au_gpt <> 'NSS') & (df.Au_gpt <> 'NIL') & \
                      (df.Au_gpt <> '<0.01') & (df.Au_gpt <> '<0.005')].astype(np.float)
    df = df.fillna({'Au_val':0.005})
elif 'Cu_ppm' in df.columns:
    df['Cu_val'] = df.Cu_ppm[(df.Cu_ppm <> ' NSS') & (df.Cu_ppm <> 'NSS') & (df.Cu_ppm <> 'NIL') & \
                      (df.Cu_ppm <> '<0.01') & (df.Cu_ppm <> '<0.005')].astype(np.float)
    df = df.fillna({'Cu_ppm':0})

In [8]:
df.index = df.Date

In [9]:
df.STD[df.STD == 'Blank'] = 'QuartzPebble'
groups = set(df['STD'])

c:\python27\arcgis10.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Generate Standard Graph

In [10]:
# with plt.xkcd():
fig = plt.figure(facecolor = 'w', figsize = (10, 12), tight_layout = False)

fig.suptitle('CRM Analyses for week of %s' % (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 16)
#fig.figimage(logo, 700,1025, zorder=1)
length = len(groups)

for a, g in enumerate(groups):   
    ax1 = fig.add_subplot(length,1,a+1)
    time_buffer = max((df.index.max() - df.index.min()) * 0.1, timedelta(5))
    datarange = [df.index.min() - time_buffer, df.index.max()+ time_buffer]

    #standard graphs
    std_mean = stds.avg[stds['alias'] == g].values[0]
    std_sd = stds.sigma[stds['alias'] == g].values[0]
    ax1.plot(datarange, [df[df.STD == g].Au_val.mean(), df[df.STD == g].Au_val.mean()], 'r-',\
              label = 'measured avg = %0.3f' % (df[df.STD == g].Au_val.mean()), linewidth = 1)
    ax1.plot(datarange, [std_mean, std_mean], 'b-', label = 'published avg = %0.3f' % (std_mean), linewidth = 1)
    _2sig1 = df[df.STD == g].Au_val.mean() + 2 * df[df.STD == g].Au_val.std()
    _2sig2 = df[df.STD == g].Au_val.mean() - 2 * df[df.STD == g].Au_val.std()
    _3sig1 = df[df.STD == g].Au_val.mean() + 3 * df[df.STD == g].Au_val.std()
    _3sig2 = df[df.STD == g].Au_val.mean() - 3 * df[df.STD == g].Au_val.std()
    _2sigm1 = std_mean + 2 * std_sd
    _2sigm2 = std_mean - 2 * std_sd
    _3sigm1 = std_mean + 3 * std_sd
    _3sigm2 = std_mean - 3 * std_sd
    ax1.plot(datarange, [_2sig1,_2sig1], 'r--', label = u'measured 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sig2,_2sig2], 'r--')
    ax1.plot(datarange, [_3sig1,_3sig1], 'r:', label = u'meausred 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sig2,_3sig2], 'r:')
    ax1.plot(datarange, [_2sigm1,_2sigm1], 'b--', label = u'published 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sigm2,_2sigm2], 'b--')
    ax1.plot(datarange, [_3sigm1,_3sigm1], 'b:', label = u'published 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sigm2,_3sigm2], 'b:', linewidth = 2)
    ax1.plot(df.Au_val[df.STD == g], 'wo', label = '_nolegend_')
    ax1.plot(df.Au_val[df.STD == g], 'k.', label = '_nolegend_')
    labels = df['SAMPLEID'][(df.STD == g) & ((df.Au_val < std_mean - 2 * std_sd) |\
                         (df.Au_val > std_mean + 2 * std_sd))]
    for d, l in labels.iteritems():
        ax1.annotate(l, xy = (d, df.Au_val[df.SAMPLEID == l]), xytext=(5, 5), textcoords= 'offset points', color = 'r')
    ylim = ax1.get_ylim()
    yshift = (ylim[1] - ylim[0]) /10.0

    ax1.set_ylim(ylim[0] - yshift, ylim[1] + yshift)
    ax1.legend(loc = 'upper left', fontsize = 'small')
    ax1.set_title('Analyses for %s, n= %d' % (g, df[df.STD == g].Au_val.count()), fontsize = 12)

    ax1.set_ylabel('Au g/t')
    xticks = ax1.get_xticks()
    date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
    ax1.set_xticklabels([])
    if a == length -1:
      ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
    
#fig.savefig('C:\\Temp\\crm.jpg', dpi = 200)  




IndexError: index 0 is out of bounds for axis 0 with size 0

Generate Blank graph

In [11]:
# with plt.xkcd():
fig = plt.figure(facecolor = 'w', figsize = (10, 8))


fig.suptitle('Blank Analyses for week of %s' % (datetime.date.today() + datetime.timedelta(4 - \
              datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 16)
#fig.figimage(logo, 690*3,1035*3, zorder=1)
length = len(groups)

for a, g in enumerate(groups): 
    ax1 = fig.add_subplot(length,1,a+1)

    time_buffer = max((df.index.max() - df.index.min()) * 0.1, timedelta(5))
    datarange = [df.index.min() - time_buffer, df.index.max()+ time_buffer]

    ax1.plot(datarange, [0.015, 0.015], 'r-', label = '3 * L.O.D. (0.015 g/t)', linewidth = 1.5)
    ax1.plot(datarange, [0.005, 0.005], 'r:', label = 'L.O.D. (0.005 g/t)', linewidth = 1.5)
    _80pct = df[df.STD == g].Au_val.quantile(0.8)
    _90pct = df[df.STD == g].Au_val.quantile(0.9)
    ax1.plot(datarange, [_80pct,_80pct], 'b--', label = u'80th percentile', linewidth = 1.5)
    ax1.plot(datarange, [_90pct,_90pct], 'b:', label = u'90th percentile', linewidth = 1.5)
    ax1.plot(df.Au_val[df.STD == g], 'wo', label = '_nolegend_')
    ax1.plot(df.Au_val[df.STD == g], 'k.', label = '_nolegend_')
    labels = df[df.STD == g].SAMPLEID[(df[df.STD == g].Au_val > 0.015)]
    for d, l in labels.iteritems():
        ax1.annotate(l, xy = (d, df.Au_val[df.SAMPLEID == l]), xytext=(5, 5), textcoords= 'offset points')
    ylim = ax1.get_ylim()
    yshift = (ylim[1] - ylim[0]) /10.0

    ax1.set_ylim(ylim[0] - yshift, ylim[1] + yshift)
    ax1.legend(loc = 'best', fontsize = 'small')
    ax1.set_title('Analyses for %s, n= %d' % (g, df[df.STD == g].Au_val.count()), fontsize = 12)

    ax1.set_ylabel('Au g/t')

    ax1.set_xticklabels([])
    if a == length -1:    
        xticks = ax1.get_xticks()
        date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
        ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
    
#fig.savefig('C:\\Temp\\blank.jpg', dpi = 200)  
